In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import os
import re
import numpy as np
from copy import deepcopy
import random
import scipy.sparse as sp
from time import time

# directory = '../MF/ml-1m'
# ratings = []
# with open(os.path.join(directory, 'ratings.dat'), encoding='latin1') as f:
#     for l in f:
#         user_id, movie_id, rating, timestamp = [int(_) for _ in l.split('::')]
#         ratings.append({
#             'user_id': user_id,
#             'movie_id': movie_id,
#             'rating': rating,
#             'timestamp': timestamp,
#             })
# ratings = pd.DataFrame(ratings)

# ratings = ratings.drop(['timestamp'],axis=1)
# print(ratings)
# # 어떤 비율로 표본 추출을 하고싶은지 df.sample에서 frac을 0~1 사이로 설정
# df = ratings.sample(frac=1).reset_index(drop=True)
# filter_user = deepcopy(df)
# counts = filter_user['user_id'].value_counts()
# filter_user = filter_user[filter_user['user_id'].isin(counts[counts >= 10].index)]

# filtered_df = deepcopy(filter_user)
# counts = filtered_df['movie_id'].value_counts()
# filtered_df = filtered_df[filtered_df['movie_id'].isin(counts[counts >=10].index)]
# filtered_df = filtered_df.reset_index(drop=True)
# print(filtered_df)


class generate_graph(object):
    def __init__(self, path):
        self.path = path  # '../NGCF-PyTorch/Data/ml-1m'
        train_file = path + '/train.txt'
        
        self.neighbor_dict = {}
        self.user, self.item = [], []
        
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    self.neighbor_dict[int(l[0])] = [int(i) for i in l[1:]]
                    self.user.append(int(l[0]))
        self.user = self.user[:1000]
        
    def jaccard_index(self, u_i, u_j, neighbor_dict):
        u_i_neighbor = self.neighbor_dict[u_i]
        u_j_neighbor = self.neighbor_dict[u_j]
        return len(list(set(u_i_neighbor) & set(u_j_neighbor))) / len(list(set(u_i_neighbor) | set(u_j_neighbor)))

    # 새로운 graph의 node j 에다가 기존 graph의 어떤 node의 neighborhood를 복사할지 zeta에 담기
    def node_copying(self):
        t1 = time()
        zeta = []
        
        for u_j in self.user:
            nor = 0
            zeta_distribution = []
            for u_i in self.user:
                nor+=self.jaccard_index(u_j, u_i, self.neighbor_dict)
            for u_m in self.user:
                zeta_distribution.append(self.jaccard_index(u_j, u_m, self.neighbor_dict) / nor)
            zeta.append(random.choices(self.user, weights=zeta_distribution)[0])
        print('total node copying time cost : ', time() - t1)
        np.save(self.path + '/zeta.npy', zeta)
        
        return zeta
    
    def generate_graph(self, epsilon, iteration):
        t2 = time()
        self.epsilon = epsilon
        self.iteration = iteration
        
        generated_node = []
        
        with open(self.path+'/sampled_graph/sampled_graph_'+str(iteration+1), 'w') as f:
            for i in self.user:
                if random.uniform(0,1) < 1-self.epsilon:  # 1-epsilon의 확률로 원래 neighbor 넣기
                    generated_node.append(i)
                else:                                # epsilon의 확률로 zeta에 있는 node의 neighbor로 copy해서 넣기
                    generated_node.append(zeta[i])
                    
                # 만들어지는 새로운 graph를 txt 파일로 저장
                f.write(str(i))
                f.write(' ')
                for j in self.neighbor_dict[generated_node[i]][:-1]:
                    f.write(str(j))
                    f.write(' ')
                f.write(str(self.neighbor_dict[generated_node[i]][-1]))
                f.write('\n')
        print('#',iteration+1,' Graph sampled time cost : ', time() - t2)

In [2]:
t = generate_graph('ml-1m')
# zeta = node_copying(user)

In [ ]:
zeta = t.node_copying()

In [3]:
zeta = np.load('ml-1m/zeta.npy')

In [4]:
# zeta를 한번 만든 후 iteration마다 generate graph하는 방식

for epoch in range(3):
    t.generate_graph(epsilon = 0.01, iteration = epoch)

# 1  Graph sampled time cost :  0.19529509544372559
# 2  Graph sampled time cost :  0.19852304458618164
# 3  Graph sampled time cost :  0.1815776824951172


In [5]:
# GAT
# 만들어지는 graph마다 embedding 초기화 
# embedding 학습하는 GNN 구조 짜기
# 학습된 embedding 가지고 x_hat (eq.11) 구하고 BPR-OPT를 maximization 시키는 방향으로 학습

In [35]:
# data로 sparse matrix 및 adjacency matrix 만들기

class Data(object):
    def __init__(self, path, batch_size):
        self.path = path
        self.batch_size = batch_size
        
        train_file = path + '/train.txt'
        # path : ml-1m
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(train_file, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ')
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            obs_adj_mat = sp.load_npz(self.path + '/obs_adj_mat.npz')
            
        except Exception:
            obs_adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/obs_adj_mat.npz', obs_adj_mat)
            
        return obs_adj_mat
    
    
    def create_adj_mat(self):
        obs_adj_mat = self.R.todok()[:1000,:]
        print('already create observed graph adjacency matrix', obs_adj_mat.shape)
        return obs_adj_mat.tocsr()
    
    # bgcf는 G_obs로부터 만들어진 sampled graphs에 대해 x hat들의 integral을 구함
    def sample(self):
        # positive / negative items 나누기
        if self.batch_size <= self.n_users:
            obs_users = random.sample(self.exist_users, self.batch_size)
        else:
            obs_users = [random.choice(self.exist_users) for _ in range(self.batch_size)]
            
        def sample_pos_items_for_u(u, num):
            # u유저의 neighbor중 num개 만큼 positive item sampling
            pos_items = self.train_items[u]
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num:
                    break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]

                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch

        def sample_neg_items_for_u(u, num):
            # u유저의 neighbor가 아닌 item 중 num개 만큼 sampling
            neg_items = []
            while True:
                if len(neg_items) == num:
                    break
                neg_id = np.random.randint(low=0, high=self.n_items, size=1)[0]
                if neg_id not in self.train_items[u] and neg_id not in neg_items:
                    neg_items.append(neg_id)
            return neg_items

    #         def sample_neg_items_for_u_from_pools(u, num):
    #             neg_items = list(set(self.neg_pools[u]) - set(self.train_items[u]))
    #             return random.sample(neg_items, num)     

        obs_pos_items, obs_neg_items = [], []
        for u in obs_users:
            obs_pos_items += sample_pos_items_for_u(u,1)
            obs_neg_items += sample_neg_items_for_u(u,1)

        return obs_users, obs_pos_items, obs_neg_items


In [7]:
# node copying으로 만들어진 graph들에 대해서 adj matrix 만들고 npz 저장하는 과정 

class sampled_graph_to_matrix(object):
    def __init__(self, path, iteration, batch_size):
        self.path =path
        self.iteration = iteration
        self.batch_size = batch_size
        
        sampled_graph = path + '/sampled_graph/sampled_graph_' + str(iteration+1)
        # path : 'ml-1m'
        
        self.n_users, self.n_items = 0, 0
        self.exist_users = []
        self.neg_pools = {}
        
        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l) > 0:
                    l = l.strip('\n').split(' ')
                    items = [int(i) for i in l[1:]]
                    uid = int(l[0])
                    self.exist_users.append(uid)
                    self.n_users = max(self.n_users, uid)
                    self.n_items = max(self.n_items, max(items))

        self.n_users+=1
        self.n_items+=1
        print(self.n_users, self.n_items)

        self.R = sp.dok_matrix((self.n_users, self.n_items), dtype=np.float32)
        self.train_items = {}

        with open(sampled_graph, 'r') as f:
            for l in f.readlines():
                if len(l)==0:
                    break
                l = l.strip('\n').split(' ') 
                uid, items = int(l[0]), [int(i) for i in l[1:]]

                for i in items:
                    self.R[uid, i] = 1

                self.train_items[uid] = items
                
    def get_adj_mat(self):
        try:
            adj_mat = sp.load_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz')
            
        except Exception:
            adj_mat = self.create_adj_mat()
            sp.save_npz(self.path + '/s_adj_mat_' + str(self.iteration+1) + '.npz', adj_mat)
            
        return adj_mat
    
     
    def create_adj_mat(self):
        adj_mat = self.R.todok()
        print('already create adjacency matrix', adj_mat.shape)
        return adj_mat.tocsr()
    
    def negative_pool(self):
        t1 = time()
        for u in self.train_items.keys():
            neg_items = list(set(range(self.n_items)) - set(self.train_items[u]))
            pools = [random.choice(neg_items) for _ in range(100)]
            self.neg_pools[u] = pools
        print('refresh negative pools', time() - t1)
    
    # bgcf는 G_obs로부터 만들어진 sampled graphs에 대해 x hat들의 integral을 구함
    def sample(self):
        # positive / negative items 나누기
        if self.batch_size <= self.n_users:
            users = random.sample(self.exist_users, self.batch_size)
        else:
            users = [random.choice(self.exist_users) for _ in range(self.batch_size)]
            
        def sample_pos_items_for_u(u, num):
            # u유저의 neighbor중 num개 만큼 positive item sampling
            pos_items = self.train_items[u]
            n_pos_items = len(pos_items)
            pos_batch = []
            while True:
                if len(pos_batch) == num:
                    break
                pos_id = np.random.randint(low=0, high=n_pos_items, size=1)[0]
                pos_i_id = pos_items[pos_id]
                
                if pos_i_id not in pos_batch:
                    pos_batch.append(pos_i_id)
            return pos_batch
        
        def sample_neg_items_for_u(u, num):
            # u유저의 neighbor가 아닌 item 중 num개 만큼 sampling
            neg_items = []
            while True:
                if len(neg_items) == num:
                    break
                neg_id = np.random.randint(low=0, high=self.n_items, size=1)[0]
                if neg_id not in self.train_items[u] and neg_id not in neg_items:
                    neg_items.append(neg_id)
            return neg_items
        
#         def sample_neg_items_for_u_from_pools(u, num):
#             neg_items = list(set(self.neg_pools[u]) - set(self.train_items[u]))
#             return random.sample(neg_items, num)     

        pos_items, neg_items = [], []
        for u in users:
            pos_items += sample_pos_items_for_u(u,1)
            neg_items += sample_neg_items_for_u(u,1)
            
        return users, pos_items, neg_items

In [31]:
obs_graph = Data(path='ml-1m', batch_size = 1000)
obs_users, obs_pos_items, obs_neg_items = obs_graph.sample()

6040 3953


In [8]:
for iteration in range(3):
    sampled_graph = sampled_graph_to_matrix(path='ml-1m', iteration = iteration, batch_size=1000)
    sampled_graph.get_adj_mat()
    users, pos_items, neg_items = sampled_graph.sample()

1000 3953
1000 3953
1000 3953


#### ngcf에서의 w_gc와 w_bi는 gc는 neighbor aggregate하는 부분의 weight, bi는 element-wise product하는 부분의 weight
#### bgcf에서는 single feed forward layer로 구성하였고 총 3개의 weight가 학습됨 (GAT에서 2개)

In [13]:
# previous version of GAT layer

class GATLayer(nn.Module):
    # TODO : 초기함수에 positive/negative item 리스트를 받아서 각 positive/negative item들의 embedding이 output으로 나오게끔
    def __init__(self, n_users, n_items, in_features, out_features, dropout = 0.2):
        super(GATLayer, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dropout = dropout
        self.in_features = in_features      # 기존 노드의 embedding 차원수 
        self.out_features = out_features    # 결과 weight의 embedding 차원수
        
        initializer = nn.init.xavier_uniform_
        
        # self.W = nn.Parameter(initializer(torch.empty(in_features, out_features)))
        # W_1은 a*e aggregate 부분, W_2는 e aggregate 부분
        self.weight_dict = nn.ParameterDict({
            'W_1' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))),
            'W_2' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))),
            'W_observed' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))) 
        })
        
        self.embedding_dict = nn.ParameterDict({
            'user_emb' : nn.Parameter(initializer(torch.empty(self.n_users, self.out_features))),
            'item_emb' : nn.Parameter(initializer(torch.empty(self.n_items, self.out_features)))
        })
        
    def create_bpr_loss(self, users, pos_items, neg_items):
        pos_scores = torch.sum(torch.mul(users, pos_items), axis=1)
        neg_scores = torch.sum(torch.mul(users, neg_items), axis=1)
        
        maxi = nn.LogSigmoid()(pos_scores - neg_scores)
        
        mf_loss = -1 * torch.mean(maxi)
        
        regularizer = (torch.norm(users) ** 2
                       + torch.norm(pos_items) ** 2
                       + torch.norm(neg_items) ** 2) / 2
        emb_loss = regularizer / self.batch_size
        #emb_loss = self.decay * regularizer / self.batch_size  (최종)
            
    
    def forward(self, users, pos_items, neg_items, adj_matrix, iteration):
        self.adj_matrix = adj_matrix
        self.iteration = iteration
        coef = torch.zeros(self.n_users, self.n_items)
        
#         adj_matrix = sp.load_npz(path + '/s_adj_mat_' + str(iteration+1) +'.npz').toarray()
#         adj_matrix = torch.Tensor(adj)
        
        user_emb = self.embedding_dict['user_emb']
        item_emb = self.embedding_dict['item_emb']
        
        # self.W의 크기를 정확하게 확인해야함, 최종적인 각 node의 embedding size 는 1 x emb_size 로 나오게끔
        # score는 각각 user-item 간의 attention score
        score = torch.exp(torch.inner(user_emb, item_emb))
        score = torch.multiply(score, adj_matrix)
        
        for i in range(score.shape[0]):
            norm = sum(score[i])
            normalized_score = score[i] / norm
            coef[i] = normalized_score
        # coef에다가 normalized된 최종적인 attention score 저장
        w_1 = self.weight_dict['W_1']
        w_2 = self.weight_dict['W_2']
        # h_tilde_1은 w_1과 곱해지는 부분의 embedding (e_k)를 user와 item으로 쪼개서 계산후 concat
        h_tilde_1_user = torch.matmul(coef, item_emb)
        h_tilde_1_user = torch.matmul(h_tilde_1_user, w_1)
        h_tilde_1_item = torch.matmul(coef.T, user_emb)
        h_tilde_1_item = torch.matmul(h_tilde_1_item, w_1)
        h_tilde_1 = torch.cat((h_tilde_1_user, h_tilde_1_item), dim=0)
        
        neighbor_num_user, neighbor_num_item = torch.zeros(self.n_users), torch.zeros(self.n_items)
        for i in range(self.n_users):
            neighbor_num_user[i] = sum(adj_matrix[i])
        for j in range(self.n_items):
            neighbor_num_item[i] = sum(adj_matrix.T[i])
        # neighbor_num은 n_j에 해당하는 부분
        
        # h_tilde_2는 w_2와 곱해지는 부분
        h_tilde_2_user = torch.matmul(adj_matrix, item_emb)
        # h_tilde_2_user = torch.matmul(neighbor_num_user, h_tilde_2_user)
        # 각 user의 neighbor의 수 만큼 item embedding에 곱해주는 과정
        for i in range(neighbor_num_user.shape[0]):
            h_tilde_2_user[i] = neighbor_num_user[i] * h_tilde_2_user[i]
        h_tilde_2_user = torch.matmul(h_tilde_2_user, w_2)
        h_tilde_2_item = torch.matmul(adj_matrix.T, user_emb)
        # h_tilde_2_item = torch.matmul(neighbor_num_item, h_tilde_2_item)
        # 각 item의 neighbor의 수 만큼 user embedding에 곱해주는 과정
        for j in range(neighbor_num_item.shape[0]):
            h_tilde_2_item[i] = neighbor_num_item[i] * h_tilde_2_item[i]
        h_tilde_2_item = torch.matmul(h_tilde_2_item, w_2)
        h_tilde_2 = torch.cat((h_tilde_2_user, h_tilde_2_item), dim=0)
        
        print(h_tilde_1.shape, h_tilde_2.shape)
        
        h_tilde_sampled = torch.cat((h_tilde_1, h_tilde_2), dim=0)
        
        # h_tilde_observed 
        
        # h_tilde_sampled를 user embedidng / pos, neg item embedding들로 쪼개야함
        return h_tilde_sampled

In [39]:
class BGCFLayer(nn.Module):
    # TODO : 초기함수에 positive/negative item 리스트를 받아서 각 positive/negative item들의 embedding이 output으로 나오게끔
    def __init__(self, n_users, n_items, in_features, out_features, dropout = 0.2):
        super(BGCFLayer, self).__init__()
        self.n_users = n_users
        self.n_items = n_items
        self.dropout = dropout
        self.in_features = in_features      # 기존 노드의 embedding 차원수 
        self.out_features = out_features    # 결과 weight의 embedding 차원수
        
        initializer = nn.init.xavier_uniform_
        
        # self.W = nn.Parameter(initializer(torch.empty(in_features, out_features)))
        # W_1은 a*e aggregate 부분, W_2는 e aggregate 부분
        self.weight_dict = nn.ParameterDict({
            'W_1' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))),
            'W_2' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features))),
            'W_obs' : nn.Parameter(initializer(torch.empty(self.in_features, self.out_features)))
        })
        
        self.embedding_dict = nn.ParameterDict({
            'user_emb' : nn.Parameter(initializer(torch.empty(self.n_users, self.out_features))),
            'item_emb' : nn.Parameter(initializer(torch.empty(self.n_items, self.out_features)))
        })
        
    def create_bpr_loss(self, users, pos_items, neg_items):
        pos_scores = torch.sum(torch.mul(users, pos_items), axis=1)
        neg_scores = torch.sum(torch.mul(users, neg_items), axis=1)
        
        maxi = nn.LogSigmoid()(pos_scores - neg_scores)
        
        mf_loss = -1 * torch.mean(maxi)
        
        regularizer = (torch.norm(users) ** 2
                       + torch.norm(pos_items) ** 2
                       + torch.norm(neg_items) ** 2) / 2
        
        emb_loss = regularizer / 1000
        # emb_loss = self.decay * regularizer / self.batch_size  (최종)
        
        mf_loss = torch.nan_to_num(mf_loss)
        emb_loss = torch.nan_to_num(emb_loss)
        
        return mf_loss+emb_loss, mf_loss, emb_loss
            
    
    def forward(self, 
                users, 
                pos_items, 
                neg_items, 
                adj_matrix, 
                obs_users, 
                obs_pos_items, 
                obs_neg_items, 
                obs_adj_matrix, 
                iteration):
        self.adj_matrix = adj_matrix
        self.iteration = iteration
        coef = torch.zeros(self.n_users, self.n_items)
        
#         adj_matrix = sp.load_npz(path + '/s_adj_mat_' + str(iteration+1) +'.npz').toarray()
#         adj_matrix = torch.Tensor(adj)
        
        user_emb = self.embedding_dict['user_emb']
        item_emb = self.embedding_dict['item_emb']
        
        # score는 각각 user-item 간의 attention score
        score = torch.exp(torch.inner(user_emb, item_emb))
        score = torch.multiply(score, adj_matrix)
        
        for i in range(score.shape[0]):
            norm = sum(score[i])
            normalized_score = score[i] / norm
            coef[i] = normalized_score
        # coef에다가 normalized된 최종적인 attention score 저장
        w_1 = self.weight_dict['W_1']
        w_2 = self.weight_dict['W_2']
        # h_tilde_1은 w_1과 곱해지는 부분의 embedding (e_k)를 user와 item으로 쪼개서 계산후 concat
        h_tilde_1_user = torch.matmul(coef, item_emb)
        h_tilde_1_user = torch.matmul(h_tilde_1_user, w_1)
        h_tilde_1_item = torch.matmul(coef.T, user_emb)
        h_tilde_1_item = torch.matmul(h_tilde_1_item, w_1)
        # h_tilde_1 = torch.cat((h_tilde_1_user, h_tilde_1_item), dim=0)
        
        neighbor_num_user, neighbor_num_item = torch.zeros(self.n_users), torch.zeros(self.n_items)
        for i in range(self.n_users):
            neighbor_num_user[i] = sum(adj_matrix[i])
        for j in range(self.n_items):
            neighbor_num_item[i] = sum(adj_matrix.T[i])
        # neighbor_num은 n_j에 해당하는 부분
        
        # h_tilde_2는 w_2와 곱해지는 부분
        h_tilde_2_user = torch.matmul(adj_matrix, item_emb)
        # h_tilde_2_user = torch.matmul(neighbor_num_user, h_tilde_2_user)
        # 각 user의 neighbor의 수 만큼 item embedding에 곱해주는 과정
        for i in range(neighbor_num_user.shape[0]):
            h_tilde_2_user[i] = (1 / neighbor_num_user[i]) * h_tilde_2_user[i]
        h_tilde_2_user = torch.matmul(h_tilde_2_user, w_2)
        h_tilde_2_item = torch.matmul(adj_matrix.T, user_emb)
        # h_tilde_2_item = torch.matmul(neighbor_num_item, h_tilde_2_item)
        # 각 item의 neighbor의 수 만큼 user embedding에 곱해주는 과정
        for j in range(neighbor_num_item.shape[0]):
            h_tilde_2_item[i] = (1 / neighbor_num_item[i]) * h_tilde_2_item[i]
        h_tilde_2_item = torch.matmul(h_tilde_2_item, w_2)
        # h_tilde_2 = torch.cat((h_tilde_2_user, h_tilde_2_item), dim=0)
        
        h_tilde_sampled_user = torch.cat((h_tilde_1_user, h_tilde_2_user), dim=1)
        h_tilde_sampled_item = torch.cat((h_tilde_2_item, h_tilde_2_item), dim=1)
        
        h_tilde_sampled_pos_item = h_tilde_sampled_item[pos_items,:]
        h_tilde_sampled_neg_item = h_tilde_sampled_item[neg_items,:]
        
        print('user embedding shape : ',h_tilde_sampled_user.shape)
        print('positive item embedding shape : ',h_tilde_sampled_pos_item.shape)
        print('negative item embedding shape : ',h_tilde_sampled_neg_item.shape)
        
        
        ### h_tilde_observed 
        w_obs = self.weight_dict['W_obs']
        
        # observed graph의 neighbor 정보 
        obs_neighbor_num_user, obs_neighbor_num_item = torch.zeros(self.n_users), torch.zeros(self.n_items)
        # 각 원소 = 각 node의 neighbor수
        for i in range(self.n_users):
            obs_neighbor_num_user[i] = sum(obs_adj_matrix[i])
        for j in range(self.n_items):
            obs_neighbor_num_item[i] = sum(obs_adj_matrix.T[i])
        # neighbor_num은 n_j에 해당하는 부분
        
        # h_tilde_obs_user
        h_tilde_obs_user = torch.matmul(obs_adj_matrix, item_emb)
        for i in range(obs_neighbor_num_user.shape[0]):
            h_tilde_obs_user[i] = (1 / obs_neighbor_num_user[i]) * h_tilde_obs_user[i]
        h_tilde_obs_user = torch.sigmoid(torch.matmul(h_tilde_obs_user, w_obs))
        
        # h_tilde_obs_item
        h_tilde_obs_item = torch.matmul(obs_adj_matrix.T, user_emb)
        for j in range(obs_neighbor_num_item.shape[0]):
            h_tilde_obs_item[i] = (1 / obs_neighbor_num_item[i]) * h_tilde_obs_item[i]
        h_tilde_obs_item = torch.sigmoid(torch.matmul(h_tilde_obs_item, w_obs))
        
        h_tilde_obs_pos_item = h_tilde_obs_item[obs_pos_items,:]
        h_tilde_obs_neg_item = h_tilde_obs_item[obs_neg_items,:]
        
        # Final embedding
        h_tilde_user = torch.sigmoid(torch.cat((h_tilde_sampled_user, h_tilde_obs_user), dim=1))
        h_tilde_pos_item = torch.sigmoid(torch.cat((h_tilde_sampled_pos_item, h_tilde_obs_pos_item), dim=1))
        h_tilde_neg_item = torch.sigmoid(torch.cat((h_tilde_sampled_neg_item, h_tilde_obs_neg_item), dim=1))
        
        # h_tilde_sampled를 user embedidng / pos, neg item embedding들로 쪼개야함
        return h_tilde_user, h_tilde_pos_item, h_tilde_neg_item

In [40]:
# pos_items, neg_items = sampled_graph_to_matrix(path='ml-1m', iteration = iteration).divide_pos_neg()
# 완성형은 GATLayer(6040, 3953, 64,64, pos_items, neg_items)
model = BGCFLayer(1000, 3953, 64, 64)

t0 = time()

optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=5e-4)

path = 'ml-1m'
for epoch in range(1):
    loss, mf_loss, emb_loss = 0., 0., 0.
    
    obs_graph = Data(path = path, batch_size = 1000)
    obs_adj_matrix = obs_graph.get_adj_mat().toarray()
    obs_adj_matrix = torch.Tensor(obs_adj_matrix)
    obs_users, obs_pos_items, obs_neg_items = obs_graph.sample()
    
    sampled_graph = sampled_graph_to_matrix(path = path, iteration = epoch, batch_size=1000)
    adj_matrix = sampled_graph.get_adj_mat().toarray()
    adj_matrix = torch.Tensor(adj_matrix)
    users, pos_items, neg_items = sampled_graph.sample()
    
    for iteration in range(1):
        t1 = time()
        
        # sampled_graph.get_adj_mat 함수로 바꿔야할듯
#         adj_matrix = sp.load_npz(path + '/s_adj_mat_' + str(iteration+1) +'.npz').toarray()
#         adj_matrix = torch.Tensor(adj_matrix)

        '''불러오는 sampled graph matrix마다 pos, neg item set을 만들고 (함수 사용) 
        bpr loss 함수에다가 각 item들에 해당하는 embedding을 입력으로 넣어줌'''

        # sampled_graph = sampled_graph_to_matrix(path = path, iteration = iteration, batch_size=1000)
        # users, pos_items, neg_items = sampled_graph.sample()
        u_g_embeddings, pos_i_g_embeddings, neg_i_g_embeddings = model(users,
                                                                       pos_items,
                                                                       neg_items,
                                                                       adj_matrix,
                                                                       obs_users,
                                                                       obs_pos_items,
                                                                       obs_neg_items,
                                                                       obs_adj_matrix,
                                                                       iteration)
        
        batch_loss, batch_mf_loss, batch_emb_loss = model.create_bpr_loss(u_g_embeddings,
                                                                          pos_i_g_embeddings,
                                                                          neg_i_g_embeddings)
        
        
        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        
        # print(batch_loss, batch_mf_loss, batch_emb_loss)
        
        loss += batch_loss
        mf_loss += batch_mf_loss
        emb_loss += batch_emb_loss
        
        print(loss, mf_loss, emb_loss)

6040 3953
1000 3953
user embedding shape :  torch.Size([1000, 128])
positive item embedding shape :  torch.Size([1000, 128])
negative item embedding shape :  torch.Size([1000, 128])
pos_scores :  tensor([59.1553, 56.8840, 56.5704, 57.6291, 57.1323, 56.7671, 57.8176, 57.8524,
        57.8459, 56.1807, 56.8830, 56.8802, 57.5251, 56.0263, 56.9701, 58.0207,
        56.8089, 56.4857, 56.3988, 57.9065, 56.7772, 56.8028, 56.1511, 56.5376,
        56.6839, 58.8255, 56.7236, 58.8100, 57.3102, 59.1374, 57.6681, 56.6598,
        56.8446, 56.9003, 56.8299, 57.2104, 57.6041, 56.6081, 57.5836, 56.8928,
        57.2445, 57.8059, 57.5111, 57.7835, 57.1465, 56.6595, 56.6108, 57.5349,
        56.1517, 57.2596, 55.9557, 57.5799, 57.0889, 56.6063, 59.2190, 56.8124,
        58.6131, 57.2274, 58.1403, 56.9822, 56.0698, 57.2879, 57.2703, 56.9146,
        56.5990, 58.4139, 56.7911, 56.6289, 57.1644, 57.2906, 57.1941, 58.3887,
        56.7106, 57.3783, 56.9993, 56.7839, 55.8838, 56.5094, 56.7225, 56.4861,
    

tensor(87.3232, grad_fn=<AddBackward0>) tensor(0.6681, grad_fn=<AddBackward0>) tensor(86.6551, grad_fn=<AddBackward0>)


In [12]:
adj = sp.load_npz('ml-1m/s_adj_mat_1.npz')
adj = adj.toarray()
print(adj.shape)
adj = torch.Tensor(adj)
coef = torch.zeros(3,3953)
user_emb = nn.Parameter(initializer(torch.empty(1000,32)))
item_emb = nn.Parameter(initializer(torch.empty(3953,32)))
w_1 = nn.Parameter(initializer(torch.empty(32,32)))
w_2 = nn.Parameter(initializer(torch.empty(32,32)))

x = torch.exp(torch.inner(user_emb, item_emb))
print(x)
x = torch.multiply(x, adj)
print(x)
for i in range(3):
    tot = sum(x[i])
    score = x[i] / tot
    coef[i] = score
    
w1 = torch.matmul(coef, item_emb)
print(w1.shape)
w1 = torch.matmul(w1, w_1)
print(w1.shape)
w2 = torch.matmul(coef.T, user_emb[:3])
print(w2.shape)
w2 = torch.matmul(w2, w_2)
print(w2.shape)
w3 = torch.cat((w1, w2), dim=0)
w3.shape

user_emb = nn.Parameter(initializer(torch.empty(1000,32)))
item_emb = nn.Parameter(initializer(torch.empty(3953,32)))
avg_emb = torch.matmul(adj.T, user_emb)
avg_emb.shape
# x = torch.multiply(x, adj)
# print(x.shape)
# x = F.softmax(x, dim=1)
# x


(1000, 3953)


NameError: name 'initializer' is not defined

In [ ]:
def get_neighbor_num(row):
    return sum(row)

In [ ]:
sumemb = torch.zeros((5))

In [ ]:
for i in range(5):
    sumemb[i] = sum(adj[i])
    
torch.matmul(sumemb, item_emb[:5]).shape
# sumemb

In [ ]:
initializer = nn.init.xavier_uniform_
a = nn.Parameter(initializer(torch.empty(5,5)))
a

In [ ]:
initializer = nn.init.xavier_uniform_
user_emb = nn.Parameter(initializer(torch.empty(5,10)))
user_emb

In [ ]:
a = np.matrix([[0,1,2],[1,1,1],[0,0,1]])
adj = np.matrix([[0,0,1],[1,0,0],[1,1,0]])
e = np.matrix([[1,2,3],[4,5,6],[7,8,9]])

print(a,'\n')
print(adj,'\n')
print(e,'\n')

a_jk = np.multiply(a,adj)
print(a_jk, '\n')

np.matmul(a_jk, e)
    
    
    